In [183]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [184]:
with open('/content/gdrive/MyDrive/Copy of sponge_bob_transcript.txt', 'r') as f:
  text = f.read()

print(text[:2000])

Mr Krabs: Where could they be? They should've been here hours ago! Arrgh! Not a customer in sight. If I don't make any money today, I surely break out in a rash!
Spongebob: Yippee! I'm rich! Look Patrick, eight gold dubloons!
Mr Krabs: Wait, I saw it first! hah! (Jumps onto table) Mine, mine! Huh?
Spongebob: Boy Mr Krabs, you sure are sweaty.
Mr Krabs: What's this? Where are the dubloons?
Spongebob: (laughs) There are no real dubloons, Mr Krabs! It's a game: 'The Flying Dutchman's Treasure Hunt'.
Patrick: Based on a real treasure map.
Spongebob: Take a break and play around with us.
Patrick: Yeah! C'mon sweaty.
Mr Krabs: Have you finished cleaning the tables?
Spongebob: I cleaned the tables Mr Krabs!
Mr Krabs: Ay, but did you scrape all the gum off the underside?
Spongebob: (chewing) I already took care of it.
Mr Krabs: Ha ha ha. All right, lads. Looks like you've shanghaeided me.
Patrick: My turn. (rolls dice) Five. (makes strawberry noise while moving piece) (reads card) One of your 

In [185]:
import numpy as np

num_lines = len(text.split('\n'))
num_unique_words = np.unique(text.split()).shape[0]

print('Number of lines = {}'.format(num_lines))
print('Number of unique words = {}'.format(num_unique_words))

Number of lines = 8819
Number of unique words = 17682


In [186]:
dict_punctuation = {
        ".":"||Period||",
        ",":"||Comma||",
        "\"":"||Quotation||",
        ";":"||SemiColon||",
        ":":"||Colon||",
        "!":"||Exclamation||",
        "?":"||QuestionMark||",
        ")":"||LeftParentheses||",
        "(":"||RightParentheses||",
        "-":"||Dash||",
        "\n":"||Return||"
    }

def encode_punctuation(text):
    '''
    Function to replace all punctuation by encoded words

    Parameters
    ----------
    text: string
      string to encode punctuation for

    Returns
    -------
    string
        The encoded string after punctutation was replaced with words 
    '''

    for key, token in dict_punctuation.items():
        text = text.replace(key, ' {} '.format(token))
    
    return text

encode_punctuation(text[:500])

"Mr Krabs ||Colon||  Where could they be ||QuestionMark||  They should've been here hours ago ||Exclamation||  Arrgh ||Exclamation||  Not a customer in sight ||Period||  If I don't make any money today ||Comma||  I surely break out in a rash ||Exclamation||  ||Return|| Spongebob ||Colon||  Yippee ||Exclamation||  I'm rich ||Exclamation||  Look Patrick ||Comma||  eight gold dubloons ||Exclamation||  ||Return|| Mr Krabs ||Colon||  Wait ||Comma||  I saw it first ||Exclamation||  hah ||Exclamation||   ||RightParentheses|| Jumps onto table ||LeftParentheses||  Mine ||Comma||  mine ||Exclamation||  Huh ||QuestionMark||  ||Return|| Spongebob ||Colon||  Boy Mr Krabs ||Comma||  you sure are sweaty ||Period||  ||Return|| Mr Krabs ||Colon||  What's this ||QuestionMark||  Where are the dubloons ||QuestionMark||  ||Return|| Spongebob ||Colon||   ||RightParentheses|| laughs ||LeftParentheses||  There are no real dubloons ||Comma||  Mr Krabs ||Exclamation||  It's a game ||Colon||  'The Flying Dutchma

In [187]:
from collections import Counter
import pprint

def create_encoding(text):
    
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return (vocab_to_int, int_to_vocab)

create_encoding(encode_punctuation(text[:500]).split())

({"'The": 71,
  'Arrgh': 26,
  'Boy': 58,
  "Dutchman's": 73,
  'Flying': 72,
  'Huh': 57,
  "Hunt'": 75,
  'I': 8,
  "I'm": 41,
  'If': 30,
  "It's": 69,
  'Jumps': 52,
  'Krabs': 4,
  'Look': 43,
  'Mine': 55,
  'Mr': 3,
  'Not': 27,
  'Patrick': 44,
  'Spongebob': 9,
  'There': 66,
  'They': 20,
  'Treasure': 74,
  'Wait': 47,
  "What's": 62,
  'Where': 12,
  'Yippee': 40,
  'a': 7,
  'ago': 25,
  'any': 33,
  'are': 11,
  'be': 19,
  'been': 22,
  'break': 37,
  'could': 17,
  'customer': 28,
  "don't": 31,
  'dubloons': 10,
  'eight': 45,
  'first': 50,
  'game': 70,
  'gold': 46,
  'hah': 51,
  'here': 23,
  'hours': 24,
  'in': 13,
  'it': 49,
  'laughs': 65,
  'make': 32,
  'mine': 56,
  'money': 34,
  'no': 67,
  'onto': 53,
  'out': 38,
  'rash': 39,
  'real': 68,
  'rich': 42,
  'saw': 48,
  "should've": 21,
  'sight': 29,
  'sure': 60,
  'surely': 36,
  'sweaty': 61,
  'table': 54,
  'the': 64,
  'they': 18,
  'this': 63,
  'today': 35,
  'you': 59,
  '||Colon||': 1,
  '||C

In [188]:
text = encode_punctuation(text)

text = text.lower().split()

vocab_to_int, int_to_vocab = create_encoding(text)
print("Number of unique words: {}".format(len(vocab_to_int)))

np.save("encodings.npy",[vocab_to_int,int_to_vocab])

text_encoded = [vocab_to_int[word] for word in text]

Number of unique words: 8028


In [189]:
import torch
from torch.utils.data import TensorDataset, DataLoader

def prepare_data(words, seq_len, batch_size):
  features, targets = [], []
  for i in range(0, len(words)-seq_len):
    features.append(words[i:i+seq_len])
    targets.append(words[i+seq_len])

  features_tensor = torch.LongTensor(features)
  targets_tensor = torch.LongTensor(targets)

  dataset = TensorDataset(features_tensor, targets_tensor)
  data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  return data_loader

In [190]:
# test my dataloader

test_text = np.arange(100)
temp_loader = prepare_data(test_text, seq_len=5, batch_size=10)

data_iter = iter(temp_loader)
sample_x, sample_y = data_iter.next()

print('Features shape: {}'.format(sample_x.shape))
print('Features')
print(sample_x)

print('*'*20)

print('Targets shape: {}'.format(sample_y.shape))
print('Targets')
print(sample_y)

Features shape: torch.Size([10, 5])
Features
tensor([[ 2,  3,  4,  5,  6],
        [62, 63, 64, 65, 66],
        [27, 28, 29, 30, 31],
        [32, 33, 34, 35, 36],
        [33, 34, 35, 36, 37],
        [67, 68, 69, 70, 71],
        [26, 27, 28, 29, 30],
        [42, 43, 44, 45, 46],
        [49, 50, 51, 52, 53],
        [50, 51, 52, 53, 54]])
********************
Targets shape: torch.Size([10])
Targets
tensor([ 7, 67, 32, 37, 38, 72, 31, 47, 54, 55])


In [191]:
# Check GPU Avilability

is_CUDA = torch.cuda.is_available()
if is_CUDA:
  print('Hello CUDA :)')
else:
  print('Hello CPU :(')

Hello CUDA :)


In [192]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers,dropout=0.5):
        '''
        Recurrent PyTorch model with an embedding layer

        Parameters
        ----------
        vocab_size:
          The number of input dimensions of the neural network (number of words in the vocabulary)
        embedding_dim:
          The size of embeddings, should you choose to use them        
        hidden_dim:
          The size of the hidden layer outputs
        dropout:
          dropout to add in between LSTM/GRU layers
        '''
        super(RNN, self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.embedd = nn.Embedding(vocab_size,embedding_dim)
        self.LSTM = nn.LSTM(embedding_dim,hidden_dim,n_layers,dropout=dropout,batch_first=True)
        self.fc = nn.Linear(hidden_dim,vocab_size)

        self.dropout = nn.Dropout(p=dropout)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network

        Parameters
        ----------
        nn_input:
          The input to the neural network
        hidden:
          The hidden state

        Returns
        -------  
          Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size = nn_input.shape[0]
        
        embedded_vector = self.embedd(nn_input)
        
        LSTM_output,hidden = self.LSTM(embedded_vector,hidden)
        
        LSTM_output = self.dropout(LSTM_output.reshape(-1,self.hidden_dim))
        
        output = self.fc(LSTM_output)
        
        output = output.reshape(batch_size, -1, self.vocab_size)
        
        # get last batch
        out = output[:, -1]
        
        # return one batch of output word scores and the hidden state
        return out,hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        Parameters
        ----------
        batch_size: int
            batch_size to use to generate hidden layer

        Returns
        -------
        tuple = (torch.FloatTensor,torch.FloatTensor), shape of each tensor = (self.n_layers,batch_size,self.hidden_dim)
            The initial hidden state containing all zeros, if using GPU, the tensors should be torch.cuda.FloatTensor (i.e. on the GPU)
        '''
        # Initialize hidden state with zero weights
        hidden = (torch.zeros(self.n_layers, batch_size, self.hidden_dim),
                  torch.zeros(self.n_layers, batch_size, self.hidden_dim))
        
        # If using GPU move hidden state to GPU
        if is_CUDA:
            hidden = hidden[0].cuda(),hidden[1].cuda()
        
        return hidden

In [193]:
def forward_back_prop(model, optimizer, criterion, inp, target, hidden):

  if is_CUDA:
      inp, target = inp.cuda(), target.cuda()
    
  hidden = tuple([each.data for each in hidden])

  optimizer.zero_grad()
    
  pred, hidden_state = model.forward(inp, hidden)
    
  loss = criterion(pred.squeeze(), target)
  loss.backward()
    
  nn.utils.clip_grad_norm_(model.parameters(), 5)
  optimizer.step()

  return loss.item(), hidden_state

In [194]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

def train_rnn(model, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    training_losses = []

    n_train_batches = len(train_loader.dataset)//batch_size

    print("Training for %d epoch(s)..." % n_epochs)
    for e in range(1, n_epochs + 1):
        
        hidden = model.init_hidden(batch_size) # Initialize hidden state

        train_loss_epoch = []
        model.train()
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            if(batch_i > n_train_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(model, optimizer, criterion, inputs, labels, hidden)

            # record loss
            batch_losses.append(loss)
            train_loss_epoch.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {}/{}  Training Loss: {}\n'.format(e, n_epochs, np.average(batch_losses)))
                batch_losses = []

        training_losses.append(np.average(train_loss_epoch))

        # Save Model
        torch.save(model,'trained_LSTM.pt')

    # Plot training and validation losses
    plt.plot(training_losses,label="Training Loss")
    plt.ylabel("CrossEntropy Loss")
    plt.xlabel("Epochs")
    plt.title("Training losses")
    plt.show()

    return model

In [195]:
##################### Hyperparameters #####################

# Data parameters
sequence_length = 30  # number of words in a sequence
batch_size = 128  # number of sequences in a batch
train_ratio = 0.8
num_train_words = int(len(text_encoded)*0.8)

# Training parameters
num_epochs = 50
lr = 8e-3

# Model parameters
vocab_size = len(vocab_to_int)
embedding_dim = 200
hidden_dim = 256
n_layers = 2  # number of LSTM Layers

# Logging parameters
show_every_n_batches = 500  # show stats for every n number of batches

In [196]:
# Create dataloader
train_loader = prepare_data(text_encoded[:num_train_words], sequence_length, batch_size)

# Create model
model = RNN(vocab_size, embedding_dim, hidden_dim, n_layers,dropout=0.5)

# Move model to GPU
if is_CUDA:
    print("Training on GPU")
    model.cuda()

# Create loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Train the model
#trained_model = train_rnn(model, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

Training on GPU


In [197]:
vocab_to_int, int_to_vocab = np.load("/content/Encodings.npy",allow_pickle=True)
trained_model = torch.load('/content/trained_LSTM.pt')

In [198]:
import torch.nn.functional as F

def generate(model, first_word_id, int_to_vocab, predict_len=100):
    '''
    Generate text using the trained model

    Parameters
    ----------
    model:

    first_word_id: int
        The integer id of the first word to use to start the text generation
    int_to_vocab: dict
        Dict of word id keys to word values
    predict_len: int
        The length of text to generate

    Returns
    -------
        string
            The generated text
    '''
    model.eval()
    
    current_seq = [first_word_id]
    
    for _ in range(predict_len):
        current_seq_tensor = torch.LongTensor(current_seq[-sequence_length:]).reshape(1,-1)
        if is_CUDA:
            current_seq_tensor = current_seq_tensor.cuda()
        
        # Initialize hidden state
        hidden = model.init_hidden(1)
        
        # Forward prop
        output, _ = model.forward(current_seq_tensor, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data.cpu()
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().reshape(-1)
        
        # select the likely next word index with some element of randomness
        p = p.numpy().reshape(-1)
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # Add the generated word to the end of the current sequence
        current_seq.append(word_i)
    
    words_generated = [int_to_vocab[word_id] for word_id in current_seq]
    gen_sentences = ' '.join(words_generated)
    
    # Replace punctuation tokens
    for key, token in dict_punctuation.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token, key)
    
    return gen_sentences

In [199]:
gen_length = 100  # Number of words to generate
prime_word = 'spongebob'  # First word in the script

generated_script = generate(model, vocab_to_int[prime_word], int_to_vocab, gen_length)
print(generated_script)

spongebob sticky package organization package rusty book alive tasty whose tub pfft blueprints krabs' rope moss idiot organization echoing rocket rocket slug rocket timeless wusty rocket slug wusty sticky lullaby stance sticky timeless goldfish *the tub blueprints package package table twins teddy twins organization blueprints lickity miles kitty prophecy prophecy moss urge mock stuck infirmary blueprints decided miles substance devised cherry intelect lawyerpants lawyerpants devilish porchlight organization organization sticky stays whose pilot wallpaper cab table cab open cab frolicked approve emerges jingle wax whose whose moss egad unclogs moss stopped echoing echoing rebuilding mr wakey whose slice stares tasty egad letters


In [200]:
from nltk.translate.bleu_score import corpus_bleu
test_text = text[int(len(text_encoded)*0.8): int(len(text_encoded)*0.8)+len(generated_script)]
score = corpus_bleu(test_text, generated_script)
print(score)

0.6958208456025856


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
